# Furuta Pendulum

In [375]:
import numpy as np
import sympy as sp
from sympy import Matrix, Function, solve, simplify, nsolve, init_printing
from math import pi
from spatialmath.base import r2q, tr2angvec, skew, sin, cos, sqrt, trinv, vex
from spatialmath import SO3, SE3, Twist3

#Function declarations
def fw_kin(angle1, angle2, frame):

    d1, d2, d3 = 1.2, 0.8, 0.8

    c1 = cos(angle1); s1 = sin(angle1)
    c2 = cos(angle2); s2 = sin(angle2)

    #Set zero orientations without joints
    R_01_zero = np.array([
        [1, 0, 0],
        [0, 0, -1],
        [0, 1, 0]
    ])

    R_12_zero = np.array([
        [1, 0, 0],
        [0, 0, -1],
        [0, 1, 0]
    ])

    R_2c_zero = np.array([
        [0, 1, 0],
        [-1, 0, 0],
        [0, 0, 1]
    ])

    #Set Rotation matrices between frames
    R_w0 = np.array([
        [c1, -s1, 0],
        [s1, c1, 0],
        [0, 0, 1]
    ])

    R_01 = R_w0 @ R_01_zero

    R_12 = np.array([
        [c2, -s2, 0],
        [s2, c2, 0],
        [0, 0, 1]
    ])@R_12_zero

    R_2c = R_2c_zero

    H_01 = SE3.Rt(R_01, [0, 0, d1])
    H_12 = SE3.Rt(R_12, [0, 0, d2])
    H_2c = SE3.Rt(R_2c, [0, 0, d3])

    if(frame == 1):
        H_final = H_01
    elif(frame == 2):
        H_final = H_01 * H_12
    elif(frame == 3):
        H_final = H_01 * H_12 * H_2c

    return H_final



#Create symbolic Matrix Representation
t = sp.symbols('t')
t_1 = sp.Function('theta_1')(t)
t_2 = sp.Function('theta_2')(t)

d1,d2,d3 = 1.2, 0.8, 0.8

sym_H01 = Matrix([
    [cos(t_1), 0, sin(t_1), 0],
    [sin(t_1), 0, -cos(t_1), 0],
    [0, 1, 0, d1],
    [0, 0, 0, 1]
])
sym_H12 = Matrix([
    [cos(t_2), 0, sin(t_2), 0],
    [sin(t_2), 0, -cos(t_2), 0],
    [0, 1, 0, d2],
    [0, 0, 0, 1]
])
sym_H2c = Matrix([
    [0, 1, 0, 0],
    [-1, 0, 0, 0],
    [0, 0, 1, d3],
    [0, 0, 0, 1]
])

sym_H0c = sym_H01 @ sym_H12 @ sym_H2c
sym_H02 = sym_H01 @ sym_H12

display(sym_H0c)
display(sym_H02)

⎡-sin(θ₁(t))  cos(θ₁(t))⋅cos(θ₂(t))  sin(θ₂(t))⋅cos(θ₁(t))  0.8⋅sin(θ₁(t)) + 0 ↪
⎢                                                                              ↪
⎢cos(θ₁(t))   sin(θ₁(t))⋅cos(θ₂(t))  sin(θ₁(t))⋅sin(θ₂(t))  0.8⋅sin(θ₁(t))⋅sin ↪
⎢                                                                              ↪
⎢     0            sin(θ₂(t))             -cos(θ₂(t))                  1.2 - 0 ↪
⎢                                                                              ↪
⎣     0                 0                      0                               ↪

↪ .8⋅sin(θ₂(t))⋅cos(θ₁(t))⎤
↪                         ⎥
↪ (θ₂(t)) - 0.8⋅cos(θ₁(t))⎥
↪                         ⎥
↪ .8⋅cos(θ₂(t))           ⎥
↪                         ⎥
↪   1                     ⎦

⎡cos(θ₁(t))⋅cos(θ₂(t))  sin(θ₁(t))   sin(θ₂(t))⋅cos(θ₁(t))  0.8⋅sin(θ₁(t)) ⎤
⎢                                                                          ⎥
⎢sin(θ₁(t))⋅cos(θ₂(t))  -cos(θ₁(t))  sin(θ₁(t))⋅sin(θ₂(t))  -0.8⋅cos(θ₁(t))⎥
⎢                                                                          ⎥
⎢     sin(θ₂(t))             0            -cos(θ₂(t))             1.2      ⎥
⎢                                                                          ⎥
⎣          0                 0                 0                   1       ⎦

## Problem 1a
> DONE


In [341]:
#set joint angles to (pi/3, -(3*pi)/7)
#frame 3 position and orientation

theta1 = (pi/3)+pi/2
theta2 = -(3*pi)/7


print("Transformation from world frame to frame 3 is")
H1a = fw_kin(theta1, theta2, 3)
print(H1a)

print("\nRotation Matrix \n", H1a.R)

print("\nQuaternions (s, k1, k2, k3)\n", r2q(H1a.R))





Transformation from world frame to frame 3 is
  -0.5      -0.1927    0.8443    1.075     
  -0.866     0.1113   -0.4875    0.3028    
   0        -0.9749   -0.2225    1.022     
   0         0         0         1         


Rotation Matrix 
 [[-0.5        -0.19270878  0.84431234]
 [-0.8660254   0.11126047 -0.48746396]
 [ 0.         -0.97492791 -0.22252093]]

Quaternions (s, k1, k2, k3)
 [ 0.3117449  -0.39091574  0.67708593 -0.53995801]


## Problem 1b
> DONE

In [342]:
#joint angles from 1a
#frame 2


print("Transformation from world frame to frame 2 is")
H1b = fw_kin(theta1, theta2, 2)
print(H1b)

print("\nRotation Matrix \n", H1b.R)


print("\nQuaternions (s, k1, k2, k3)\n", r2q(H1b.R))





Transformation from world frame to frame 2 is
  -0.1927    0.5       0.8443    0.4       
   0.1113    0.866    -0.4875    0.6928    
  -0.9749    0        -0.2225    1.2       
   0         0         0         1         


Rotation Matrix 
 [[-0.19270878  0.5         0.84431234]
 [ 0.11126047  0.8660254  -0.48746396]
 [-0.97492791  0.         -0.22252093]]

Quaternions (s, k1, k2, k3)
 [ 0.6022449   0.20235288  0.75519122 -0.16137104]


## Problem 1c
> DONE

In [343]:
#axis angle representation
#rodriguez formula

angle_1a, axis_1a = tr2angvec(H1a.R)
print("\nAxis Angle Representation 1a (angle, k1, k2, k3 )\n", angle_1a, axis_1a)

angle_1b, axis_1b = tr2angvec(H1b.R)
print("\nAxis Angle Representation 1b (angle, k1, k2, k3 )\n", angle_1b, axis_1b)

I = SO3()

Rod_1a = I + sin(angle_1a)*skew(axis_1a) + (1 - cos(angle_1a))*(skew(axis_1a)@skew(axis_1a))

Rod_1b = I + sin(angle_1b)*skew(axis_1b) + (1 - cos(angle_1b))*(skew(axis_1b)@skew(axis_1b))

print("\n1a recovered rotation matrix from rodriguez formula\n", Rod_1a.round(4))

print("\n1b recovered rotation matrix from rodriguez formula\n", Rod_1b.round(4))



Axis Angle Representation 1a (angle, k1, k2, k3 )
 2.5075348579646866 [-0.41141843  0.71259763 -0.56827765]

Axis Angle Representation 1b (angle, k1, k2, k3 )
 1.8489722552687557 [ 0.25347612  0.94598578 -0.20214046]

1a recovered rotation matrix from rodriguez formula
 [[-0.5    -0.1927  0.8443]
 [-0.866   0.1113 -0.4875]
 [ 0.     -0.9749 -0.2225]]

1b recovered rotation matrix from rodriguez formula
 [[-0.1927  0.5     0.8443]
 [ 0.1113  0.866  -0.4875]
 [-0.9749  0.     -0.2225]]


## Problem 1d
> DONE

In [344]:
k_1d = Matrix([-0.5, 0.5, -sqrt(2)/2])
z_1d = Matrix([sym_H0c[0,2], sym_H0c[1,2], sym_H0c[2,2]])



eqns_1d = z_1d - k_1d
eqns_1d = eqns_1d.subs({t_1: 'theta_1'})
eqns_1d = eqns_1d.subs({t_2: 'theta_2'})


solutions_1d = solve(eqns_1d, ('theta_1', 'theta_2'), dict=True)

sp.pprint(eqns_1d)

print("\nFinal Solution 1\n")
sp.pprint(solutions_1d[0])
print("\nFinal Solution 2\n")
sp.pprint(solutions_1d[1])

⎡   sin(θ₂)⋅cos(θ₁) + 0.5   ⎤
⎢                           ⎥
⎢   sin(θ₁)⋅sin(θ₂) - 0.5   ⎥
⎢                           ⎥
⎣0.707106781186548 - cos(θ₂)⎦

Final Solution 1

{θ₁: -0.785398163397449, θ₂: 5.49778714378214}

Final Solution 2

{θ₁: 2.35619449019234, θ₂: 0.785398163397448}


## Problem 1e
> DONE

In [345]:
k_1e = Matrix([1.075, 0.303, 1.022])
d_1e = Matrix([sym_H0c[0,3], sym_H0c[1,3], sym_H0c[2,3]])

theta_1, theta_2 = sp.symbols('theta_1 theta_2')

eq1_1e = d_1e[0].subs({t_1: theta_1})
eq1_1e = eq1_1e.subs({t_2: theta_2})
Eq1_1e = sp.Eq(eq1_1e, k_1e[0])

eq2_1e = d_1e[1].subs({t_1: theta_1})
eq2_1e = eq2_1e.subs({t_2: theta_2})
Eq2_1e = sp.Eq(eq2_1e, k_1e[1])

eq3_1e = d_1e[2].subs({t_2: theta_2})
Eq3_1e = sp.Eq(eq3_1e, k_1e[2])


solution_1e = solve([Eq1_1e, Eq2_1e, Eq3_1e], (theta_1, theta_2), check=False)

solution_1e_labeled = solve([Eq1_1e, Eq2_1e, Eq3_1e], (theta_1, theta_2), check=False, dict=True)

for i in range(0, 4):
    sol = solution_1e[i]
    M = Matrix([
        [eq1_1e, k_1e[0]],
        [eq2_1e, k_1e[1]],
        [eq3_1e, k_1e[2]]
    ])
    test = M.subs(theta_1, sol[0])
    test = test.subs(theta_2, sol[1])
    print("\nSolution ",i+1,"\n")
    sp.pprint(test)

final_solutions_1e = [solution_1e_labeled[2], solution_1e_labeled[3]]

print("\nFinal Solution 1\n")
sp.pprint(final_solutions_1e[0])
print("\nFinal Solution 2\n")
sp.pprint(final_solutions_1e[1])



Solution  1 

⎡-1.07541015431323  1.075⎤
⎢                        ⎥
⎢      0.303        0.303⎥
⎢                        ⎥
⎣      1.022        1.022⎦

Solution  2 

⎡-1.07541015431323  1.075⎤
⎢                        ⎥
⎢      0.303        0.303⎥
⎢                        ⎥
⎣      1.022        1.022⎦

Solution  3 

⎡1.07541015431323  1.075⎤
⎢                       ⎥
⎢     0.303        0.303⎥
⎢                       ⎥
⎣     1.022        1.022⎦

Solution  4 

⎡1.07541015431323  1.075⎤
⎢                       ⎥
⎢     0.303        0.303⎥
⎢                       ⎥
⎣     1.022        1.022⎦

Final Solution 1

{θ₁: 1.07272370972679, θ₂: 1.34641832379787}

Final Solution 2

{θ₁: 2.61813591673788, θ₂: 4.93676698338171}


## Problem 2a
> DONE

In [382]:
#set dtheta 1 and 2 for problem
dtheta1 = 1
dtheta2 = 2


#Calculate inverse and derivative of Transformation Matrix
dsym_H0c = sp.diff(sym_H0c, t)
inv_sym_H0c = trinv(np.array(sym_H0c))
inv_sym_H0c = Matrix(inv_sym_H0c)


#Spatial Twist
spatial_twist_2a = simplify(dsym_H0c @ inv_sym_H0c)


spatial_omega_2a = spatial_twist_2a[0:3, 0:3]
spatial_omega_2a = Matrix(vex(np.array(spatial_omega_2a)))

spatial_vel_2a = spatial_twist_2a[0:3, 3]

#Twist 6x1 Vector Form
spatial_twistv_2a = Matrix([
    [spatial_vel_2a],
    [spatial_omega_2a]
])
spatial_twistv_2a_answer = spatial_twistv_2a.subs({sp.Derivative(t_1, t): dtheta1}).subs({t_1: theta1}) #This is how to substitute at the end
spatial_twistv_2a_answer = simplify(spatial_twistv_2a_answer.subs({sp.Derivative(t_2, t): dtheta2}).subs({t_2: theta2}))

#Body Twist
body_twist_2a = simplify(inv_sym_H0c @ dsym_H0c)

body_omega_2a = body_twist_2a[0:3, 0:3]
body_omega_2a = Matrix(vex(np.array(body_omega_2a)))

body_vel_2a = body_twist_2a[0:3, 3]

#Twist 6x1 Vector Form
body_twistv_2a = Matrix([
    [body_vel_2a],
    [body_omega_2a]
])
body_twistv_2a_answer = body_twistv_2a.subs({sp.Derivative(t_1, t): dtheta1}).subs({t_1: theta1})
body_twistv_2a_answer = simplify(body_twistv_2a_answer.subs({sp.Derivative(t_2, t): dtheta2}).subs({t_2: theta2}))

print("Spatial Twist 2a")
display(spatial_twistv_2a_answer)
print("Body Twist 2a")
display(body_twistv_2a_answer)

display(spatial_twistv_2a)
display(body_twistv_2a)



Spatial Twist 2a


⎡-2.07846096908265⎤
⎢                 ⎥
⎢       1.2       ⎥
⎢                 ⎥
⎢        0        ⎥
⎢                 ⎥
⎢       1.0       ⎥
⎢                 ⎥
⎢1.73205080756888 ⎥
⎢                 ⎥
⎣        1        ⎦

Body Twist 2a


⎡-0.779942329745459⎤
⎢                  ⎥
⎢ 1.77801674716505 ⎥
⎢                  ⎥
⎢-0.779942329745459⎥
⎢                  ⎥
⎢        -2        ⎥
⎢                  ⎥
⎢-0.974927912181824⎥
⎢                  ⎥
⎣-0.222520933956314⎦

⎡               d        ⎤
⎢1.2⋅cos(θ₁(t))⋅──(θ₂(t))⎥
⎢               dt       ⎥
⎢                        ⎥
⎢               d        ⎥
⎢1.2⋅sin(θ₁(t))⋅──(θ₂(t))⎥
⎢               dt       ⎥
⎢                        ⎥
⎢           0            ⎥
⎢                        ⎥
⎢             d          ⎥
⎢  sin(θ₁(t))⋅──(θ₂(t))  ⎥
⎢             dt         ⎥
⎢                        ⎥
⎢             d          ⎥
⎢ -cos(θ₁(t))⋅──(θ₂(t))  ⎥
⎢             dt         ⎥
⎢                        ⎥
⎢       d                ⎥
⎢       ──(θ₁(t))        ⎥
⎣       dt               ⎦

⎡                       d                ⎤
⎢        0.8⋅sin(θ₂(t))⋅──(θ₁(t))        ⎥
⎢                       dt               ⎥
⎢                                        ⎥
⎢               d               d        ⎥
⎢0.8⋅cos(θ₂(t))⋅──(θ₁(t)) + 0.8⋅──(θ₂(t))⎥
⎢               dt              dt       ⎥
⎢                                        ⎥
⎢                       d                ⎥
⎢        0.8⋅sin(θ₂(t))⋅──(θ₁(t))        ⎥
⎢                       dt               ⎥
⎢                                        ⎥
⎢                d                       ⎥
⎢               -──(θ₂(t))               ⎥
⎢                dt                      ⎥
⎢                                        ⎥
⎢                     d                  ⎥
⎢          sin(θ₂(t))⋅──(θ₁(t))          ⎥
⎢                     dt                 ⎥
⎢                                        ⎥
⎢                     d                  ⎥
⎢         -cos(θ₂(t))⋅──(θ₁(t))          ⎥
⎣                     dt                 ⎦

## Problem 2b
> DONE

In [347]:
#Calculate inverse and derivative of Transformation Matrix
dsym_H02 = sp.diff(sym_H02, t)
inv_sym_H02 = trinv(np.array(sym_H02))
inv_sym_H02 = Matrix(inv_sym_H02)

#Spatial Twist
spatial_twist_2b = simplify(dsym_H02 @ inv_sym_H02)

spatial_omega_2b = spatial_twist_2b[0:3, 0:3]
spatial_omega_2b = Matrix(vex(np.array(spatial_omega_2b)))

spatial_vel_2b = spatial_twist_2b[0:3, 3]

#Twist 6x1 Vector Form
spatial_twistv_2b = Matrix([
    [spatial_vel_2b],
    [spatial_omega_2b]
])
spatial_twistv_2b_answer = spatial_twistv_2b.subs({sp.Derivative(t_1, t): dtheta1}).subs({t_1: theta1}) #This is how to substitutde at the end
spatial_twistv_2b_answer = simplify(spatial_twistv_2b_answer.subs({sp.Derivative(t_2, t): dtheta2}).subs({t_2: theta2}))

#Body Twist
body_twist_2b = simplify(inv_sym_H02 @ dsym_H02)

body_omega_2b = body_twist_2b[0:3, 0:3]
body_omega_2b = Matrix(vex(np.array(body_omega_2b)))

body_vel_2b = body_twist_2b[0:3, 3]

#Twist 6x1 Vector Form
body_twistv_2b = Matrix([
    [body_vel_2b],
    [body_omega_2b]
])
body_twistv_2b_answer = body_twistv_2b.subs({sp.Derivative(t_1, t): dtheta1}).subs({t_1: theta1})
body_twistv_2b_answer = simplify(body_twistv_2b_answer.subs({sp.Derivative(t_2, t): dtheta2}).subs({t_2: theta2}))

print("Spatial Twist 2b\n")
sp.pprint(spatial_twistv_2b_answer)
print("Body Twist 2b\n")
sp.pprint(body_twistv_2b_answer)

display(spatial_twistv_2b)
display(body_twistv_2b)


Spatial Twist 2b

⎡-2.07846096908265⎤
⎢                 ⎥
⎢       1.2       ⎥
⎢                 ⎥
⎢        0        ⎥
⎢                 ⎥
⎢       1.0       ⎥
⎢                 ⎥
⎢1.73205080756888 ⎥
⎢                 ⎥
⎣        1        ⎦
Body Twist 2b

⎡0.178016747165052 ⎤
⎢                  ⎥
⎢        0         ⎥
⎢                  ⎥
⎢-0.779942329745459⎥
⎢                  ⎥
⎢-0.974927912181824⎥
⎢                  ⎥
⎢        2         ⎥
⎢                  ⎥
⎣-0.222520933956314⎦


⎡               d        ⎤
⎢1.2⋅cos(θ₁(t))⋅──(θ₂(t))⎥
⎢               dt       ⎥
⎢                        ⎥
⎢               d        ⎥
⎢1.2⋅sin(θ₁(t))⋅──(θ₂(t))⎥
⎢               dt       ⎥
⎢                        ⎥
⎢           0            ⎥
⎢                        ⎥
⎢             d          ⎥
⎢  sin(θ₁(t))⋅──(θ₂(t))  ⎥
⎢             dt         ⎥
⎢                        ⎥
⎢             d          ⎥
⎢ -cos(θ₁(t))⋅──(θ₂(t))  ⎥
⎢             dt         ⎥
⎢                        ⎥
⎢       d                ⎥
⎢       ──(θ₁(t))        ⎥
⎣       dt               ⎦

⎡               d        ⎤
⎢0.8⋅cos(θ₂(t))⋅──(θ₁(t))⎥
⎢               dt       ⎥
⎢                        ⎥
⎢           0            ⎥
⎢                        ⎥
⎢               d        ⎥
⎢0.8⋅sin(θ₂(t))⋅──(θ₁(t))⎥
⎢               dt       ⎥
⎢                        ⎥
⎢             d          ⎥
⎢  sin(θ₂(t))⋅──(θ₁(t))  ⎥
⎢             dt         ⎥
⎢                        ⎥
⎢       d                ⎥
⎢       ──(θ₂(t))        ⎥
⎢       dt               ⎥
⎢                        ⎥
⎢             d          ⎥
⎢ -cos(θ₂(t))⋅──(θ₁(t))  ⎥
⎣             dt         ⎦

## Problem 2c
> DONE

In [348]:
#Given velocity and omega in form (v,omega)
V_given_2c = Matrix([0.39, 0.871, 0.39, -1.2, 0.487, 0.111])
body_twist_2c = body_twistv_2a

init_printing()

dtheta_1 = sp.symbols(r'\dot{\theta_1}')
dtheta_2 = sp.symbols(r'\dot{\theta_2}')

body_twist_2c = body_twist_2c.subs({sp.Derivative(t_1, t): dtheta_1}).subs({t_1: 'theta_1'})

body_twist_2c = body_twist_2c.subs({sp.Derivative(t_2, t): dtheta_2}).subs({t_2: 'theta_2'})

print("\n")
display(body_twist_2c, V_given_2c)
eqns_2c = simplify(body_twist_2c.subs('theta_2', theta2))


Eqns_2c = eqns_2c - V_given_2c

solutions_2c = solve((Eqns_2c[3], Eqns_2c[0]), (dtheta_1, dtheta_2), check=False, dict=True)


display(solutions_2c)

sol1_2c = solutions_2c[0][dtheta_1]
sol2_2c = solutions_2c[0][dtheta_2]

# Testing Solution

testM_2c = eqns_2c
testM_2c = testM_2c.subs(dtheta_1, sol1_2c)
testM_2c = testM_2c.subs(dtheta_2, sol2_2c)
print("\nSolution Test")
display(testM_2c, V_given_2c)


⎡          0.8⋅\dot{\theta_1}⋅sin(θ₂)           ⎤
⎢                                               ⎥
⎢0.8⋅\dot{\theta_1}⋅cos(θ₂) + 0.8⋅\dot{\theta_2}⎥
⎢                                               ⎥
⎢          0.8⋅\dot{\theta_1}⋅sin(θ₂)           ⎥
⎢                                               ⎥
⎢                -\dot{\theta_2}                ⎥
⎢                                               ⎥
⎢            \dot{\theta_1}⋅sin(θ₂)             ⎥
⎢                                               ⎥
⎣            -\dot{\theta_1}⋅cos(θ₂)            ⎦

⎡0.39 ⎤
⎢     ⎥
⎢0.871⎥
⎢     ⎥
⎢0.39 ⎥
⎢     ⎥
⎢-1.2 ⎥
⎢     ⎥
⎢0.487⎥
⎢     ⎥
⎣0.111⎦


Solution Test


⎡      0.39       ⎤
⎢                 ⎥
⎢0.870985044987841⎥
⎢                 ⎥
⎢      0.39       ⎥
⎢                 ⎥
⎢      -1.2       ⎥
⎢                 ⎥
⎢     0.4875      ⎥
⎢                 ⎥
⎣0.111268693765198⎦

⎡0.39 ⎤
⎢     ⎥
⎢0.871⎥
⎢     ⎥
⎢0.39 ⎥
⎢     ⎥
⎢-1.2 ⎥
⎢     ⎥
⎢0.487⎥
⎢     ⎥
⎣0.111⎦

## Problem 2d
> DONE

In [349]:
#Given velocity and omega in form (v,omega)
V_given_2d = Matrix([-1.247, 0.72, 0, 0.6, 1.039, 0.5])
spatial_twist_2d = spatial_twistv_2a


spatial_twist_2d = spatial_twist_2d.subs({sp.Derivative(t_1, t): dtheta_1}).subs({t_1: 'theta_1'})

spatial_twist_2d = spatial_twist_2d.subs({sp.Derivative(t_2, t): dtheta_2}).subs({t_2: 'theta_2'})

print("\n")
display(spatial_twist_2d, V_given_2d)
eqns_2d = simplify(spatial_twist_2d.subs('theta_1', theta1))


Eqns_2d = eqns_2d - V_given_2d

display(eqns_2d)

solutions_2d = solve((Eqns_2d[5], Eqns_2d[1]), (dtheta_1, dtheta_2), check=False, dict=True)

display(solutions_2d)

sol1_2d = solutions_2d[0][dtheta_1]
sol2_2d = solutions_2d[0][dtheta_2]

# Testing Solution

testM_2d = eqns_2d
testM_2d = testM_2d.subs(dtheta_1, sol1_2d)
testM_2d = testM_2d.subs(dtheta_2, sol2_2d)
print("\nSolution Test")
display(testM_2d, V_given_2d)


⎡1.2⋅\dot{\theta_2}⋅cos(θ₁)⎤
⎢                          ⎥
⎢1.2⋅\dot{\theta_2}⋅sin(θ₁)⎥
⎢                          ⎥
⎢            0             ⎥
⎢                          ⎥
⎢  \dot{\theta_2}⋅sin(θ₁)  ⎥
⎢                          ⎥
⎢ -\dot{\theta_2}⋅cos(θ₁)  ⎥
⎢                          ⎥
⎣      \dot{\theta_1}      ⎦

⎡-1.247⎤
⎢      ⎥
⎢ 0.72 ⎥
⎢      ⎥
⎢  0   ⎥
⎢      ⎥
⎢ 0.6  ⎥
⎢      ⎥
⎢1.039 ⎥
⎢      ⎥
⎣ 0.5  ⎦

⎡-1.03923048454133⋅\dot{\theta_2}⎤
⎢                                ⎥
⎢       0.6⋅\dot{\theta_2}       ⎥
⎢                                ⎥
⎢               0                ⎥
⎢                                ⎥
⎢       0.5⋅\dot{\theta_2}       ⎥
⎢                                ⎥
⎢0.866025403784438⋅\dot{\theta_2}⎥
⎢                                ⎥
⎣         \dot{\theta_1}         ⎦


Solution Test


⎡-1.24707658144959⎤
⎢                 ⎥
⎢0.720000000000001⎥
⎢                 ⎥
⎢        0        ⎥
⎢                 ⎥
⎢       0.6       ⎥
⎢                 ⎥
⎢1.03923048454133 ⎥
⎢                 ⎥
⎣       0.5       ⎦

⎡-1.247⎤
⎢      ⎥
⎢ 0.72 ⎥
⎢      ⎥
⎢  0   ⎥
⎢      ⎥
⎢ 0.6  ⎥
⎢      ⎥
⎢1.039 ⎥
⎢      ⎥
⎣ 0.5  ⎦

⎡            d                                  d                              ↪
⎢-cos(θ₁(t))⋅──(θ₁(t))  - sin(θ₁(t))⋅cos(θ₂(t))⋅──(θ₁(t)) - sin(θ₂(t))⋅cos(θ₁( ↪
⎢            dt                                 dt                             ↪
⎢                                                                              ↪
⎢            d                                  d                              ↪
⎢-sin(θ₁(t))⋅──(θ₁(t))  - sin(θ₁(t))⋅sin(θ₂(t))⋅──(θ₂(t)) + cos(θ₁(t))⋅cos(θ₂( ↪
⎢            dt                                 dt                             ↪
⎢                                                                              ↪
⎢                                                         d                    ↪
⎢          0                                   cos(θ₂(t))⋅──(θ₂(t))            ↪
⎢                                                         dt                   ↪
⎢                                                                              ↪
⎣          0                

⎡                        d                                 d                   ↪
⎢- sin(θ₁(t))⋅cos(θ₂(t))⋅──(θ₁(t)) - sin(θ₂(t))⋅cos(θ₁(t))⋅──(θ₂(t))  cos(θ₁(t ↪
⎢                        dt                                dt                  ↪
⎢                                                                              ↪
⎢                        d                                 d                   ↪
⎢- sin(θ₁(t))⋅sin(θ₂(t))⋅──(θ₂(t)) + cos(θ₁(t))⋅cos(θ₂(t))⋅──(θ₁(t))  sin(θ₁(t ↪
⎢                        dt                                dt                  ↪
⎢                                                                              ↪
⎢                                  d                                           ↪
⎢                       cos(θ₂(t))⋅──(θ₂(t))                                   ↪
⎢                                  dt                                          ↪
⎢                                                                              ↪
⎣                           